In [1]:
import pandas as pd 
import process
import numpy as np 
# Jerome path : r'C:\Users\33640\OneDrive\Documents\GitHub\Portfolio_clustering_project\Data\DataBase.csv'
# Nail path : '/Users/khelifanail/Documents/GitHub/Portfolio_clustering_project/Data/DataBase.csv'
df = pd.read_csv(r'/Users/khelifanail/Documents/GitHub/Portfolio_clustering_project/Data/DataBase.csv')

df.set_index('ticker', inplace=True)

df.columns = pd.to_datetime(df.columns.str[1:], format='%Y%m%d').strftime('%d/%m/%Y')

df_cleaned = df.fillna(0) # Utilisez la méthode fillna(0) pour remplacer les NaN par 0

df_cleaned = df_cleaned.transpose() ## WE WANT COLUMNS TO BE VECTOR OF RETURN FOR A GIVEN TICKER

In [4]:
from PyFolioC import PyFolio
from PyFolioC import PyFolioC

historical_data = df_cleaned
number_of_repetitions = 10
lookback_window = [0, 50]
evaluation_window = 1
number_of_clusters = 38
clustering_method = 'SPONGE'
sigma = 0.1
eta = 0.1

consolidated_W = PyFolioC(number_of_repetitions=number_of_repetitions, historical_data=df_cleaned, lookback_window=lookback_window, evaluation_window=evaluation_window, number_of_clusters=number_of_clusters, sigma=sigma, eta=eta, clustering_method=clustering_method)

In [6]:
number_of_windows = 10

consolidated_W.sliding_window(number_of_window=number_of_windows)

AttributeError: 'PyFolioC' object has no attribute 'lookback_window_0'